In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import load_data
df = load_data.load_county_level()
df = df.sort_values(load_data.outcome_deaths, ascending=False)
print(df.shape)

loading county level data...


/accounts/projects/vision/chandan/covid-19-geographic-risk-prediction/functions/merge_data.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  diabetes = pd.read_csv(diabetes, skiprows = 2, skipfooter = 1)


ks Index(['countyFIPS', 'Respiratory Mortality'], dtype='object')
caching to data/df_county_level_cached.pkl
(1212, 7306)
